In [ ]:
#Yash Deshmukh
#yvd2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/augmented_frames"

paper  rock  scissors


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
import os

# Set random seed for reproducibility
tf.random.set_seed(42)

# Data loading and preprocessing
def load_data(data_dir, img_height=224, img_width=224):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=(img_height, img_width),
        batch_size=32
    )

    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=(img_height, img_width),
        batch_size=32
    )

    class_names = dataset.class_names
    print('Classes:', class_names)

    # Configure dataset for performance
    AUTOTUNE = tf.data.AUTOTUNE
    dataset = dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

    return dataset, val_dataset, class_names

# Create the CNN model
def create_model():
    model = models.Sequential([
        # Data augmentation layers
        layers.RandomFlip("horizontal"),
        layers.RandomRotation(0.2),
        layers.RandomZoom(0.2),

        # Convolutional layers
        layers.Conv2D(64, 3, padding='same', activation='relu', input_shape=(224, 224, 3)),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(128, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        layers.Conv2D(256, 3, padding='same', activation='relu'),
        layers.BatchNormalization(),
        layers.MaxPooling2D(),

        # Dense layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.3),
        layers.Dense(3, activation='softmax')
    ])

    return model

    return model

def main():
    # Load data
    data_dir = "/content/drive/MyDrive/Colab Notebooks/augmented_frames"
    train_dataset, val_dataset, class_names = load_data(data_dir)

    # Create and compile model
    model = create_model()

    initial_learning_rate = 0.001

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    # Model summary
    model.summary()

    # Training callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=15,
            restore_best_weights=True,
            min_delta=0.01  # Minimum change to count as an improvement
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=2,
            min_lr=0.00001
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            monitor='val_accuracy',
            save_best_only=True
        )
    ]

    # Train the model
    epochs = 40
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=epochs,
        callbacks=callbacks
    )

    # Save the final model
    model.save('final_model.h5')

    print("Training completed!")
    print(f"Final training accuracy: {history.history['accuracy'][-1]:.4f}")
    print(f"Final validation accuracy: {history.history['val_accuracy'][-1]:.4f}")

if __name__ == "__main__":
    main()

Found 2250 files belonging to 3 classes.
Using 1800 files for training.
Found 2250 files belonging to 3 classes.
Using 450 files for validation.
Classes: ['paper', 'rock', 'scissors']


Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ random_flip_3 (RandomFlip)           │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_rotation_3 (RandomRotation)   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ random_zoom_3 (RandomZoom)           │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_9 (Conv2D)                    │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_15               │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_9 (MaxPooling2D)       │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_10 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_16               │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_10 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_11 (Conv2D)                   │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_17               │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_11 (MaxPooling2D)      │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_3 (Flatten)                  │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_9 (Dense)                      │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_18               │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_6 (Dropout)                  │ ?                           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_10 (Dense)                     │ ?                           │     0 (unbuilt) │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_19               │ ?                           │     0 (unbuilt) │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_7 (Dropout)                  │ ?                           │              

 Total params: 0 (0.00 B)

 Trainable params: 0 (0.00 B)

...

57/57 ━━━━━━━━━━━━━━━━━━━━ 58s 1s/step - accuracy: 0.8208 - loss: 0.4355 - val_accuracy: 0.7489 - val_loss: 0.7998 - learning_rate: 6.2500e-05
Epoch 17/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.8282 - loss: 0.4222

57/57 ━━━━━━━━━━━━━━━━━━━━ 94s 1s/step - accuracy: 0.8281 - loss: 0.4222 - val_accuracy: 0.7622 - val_loss: 0.7875 - learning_rate: 3.1250e-05
Epoch 18/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 16s 273ms/step - accuracy: 0.8376 - loss: 0.3804 - val_accuracy: 0.6556 - val_loss: 0.9305 - learning_rate: 3.1250e-05
Epoch 19/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 16s 272ms/step - accuracy: 0.8431 - loss: 0.3927 - val_accuracy: 0.7378 - val_loss: 0.8856 - learning_rate: 1.5625e-05
Epoch 20/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 274ms/step - accuracy: 0.8249 - loss: 0.4375 - val_accuracy: 0.7578 - val_loss: 0.9708 - learning_rate: 1.5625e-05
Epoch 21/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 16s 274ms/step - accuracy: 0.8245 - loss: 0.4049 - val_accuracy: 0.7467 - val_loss: 1.1189 - learning_rate: 1.0000e-05
Epoch 22/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 264ms/step - accuracy: 0.8397 - loss: 0.3891

57/57 ━━━━━━━━━━━━━━━━━━━━ 41s 731ms/step - accuracy: 0.8398 - loss: 0.3889 - val_accuracy: 0.7778 - val_loss: 0.9689 - learning_rate: 1.0000e-05
Epoch 23/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 262ms/step - accuracy: 0.8410 - loss: 0.3947

57/57 ━━━━━━━━━━━━━━━━━━━━ 71s 1s/step - accuracy: 0.8409 - loss: 0.3950 - val_accuracy: 0.7800 - val_loss: 0.9292 - learning_rate: 1.0000e-05
Epoch 24/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 16s 274ms/step - accuracy: 0.8562 - loss: 0.3741 - val_accuracy: 0.7689 - val_loss: 0.9405 - learning_rate: 1.0000e-05
Epoch 25/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 273ms/step - accuracy: 0.8385 - loss: 0.3757 - val_accuracy: 0.7444 - val_loss: 0.9851 - learning_rate: 1.0000e-05
Epoch 26/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 21s 275ms/step - accuracy: 0.8616 - loss: 0.3482 - val_accuracy: 0.7022 - val_loss: 1.0668 - learning_rate: 1.0000e-05
Epoch 27/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 20s 274ms/step - accuracy: 0.8411 - loss: 0.3722 - val_accuracy: 0.7756 - val_loss: 0.9689 - learning_rate: 1.0000e-05
Epoch 28/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 16s 275ms/step - accuracy: 0.8410 - loss: 0.3744 - val_accuracy: 0.7489 - val_loss: 1.0032 - learning_rate: 1.0000e-05
Epoch 29/40
57/57 ━━━━━━━━━━━━━━━━━━━━ 16s 276ms/step - accuracy: 0

Training completed!
Final training accuracy: 0.8578
Final validation accuracy: 0.7378


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models, applications
import numpy as np
import os

# Set random seed for reproducibility
tf.random.set_seed(42)

# Data loading and preprocessing
def load_data(data_dir, img_height=224, img_width=224, batch_size=32):
    dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="training",
        seed=42,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    val_dataset = tf.keras.preprocessing.image_dataset_from_directory(
        data_dir,
        validation_split=0.2,
        subset="validation",
        seed=42,
        image_size=(img_height, img_width),
        batch_size=batch_size
    )

    class_names = dataset.class_names
    print('Classes:', class_names)

    # Configure dataset for performance
    AUTOTUNE = tf.data.AUTOTUNE
    dataset = dataset.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
    val_dataset = val_dataset.cache().prefetch(buffer_size=AUTOTUNE)

    return dataset, val_dataset, class_names

# Create the CNN model using transfer learning
def create_model(input_shape, num_classes):
    base_model = applications.EfficientNetB0(weights='imagenet', include_top=False, input_shape=input_shape)
    base_model.trainable = False

    inputs = layers.Input(shape=input_shape)
    x = base_model(inputs, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(512, activation='relu')(x)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = models.Model(inputs, outputs)
    return model, base_model

def main():
    # Load data
    data_dir = "/content/drive/MyDrive/Colab Notebooks/augmented_frames"
    train_dataset, val_dataset, class_names = load_data(data_dir, batch_size=64)

    # Create and compile model
    input_shape = (224, 224, 3)
    num_classes = len(class_names)
    model, base_model = create_model(input_shape, num_classes)

    initial_learning_rate = 0.001

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=initial_learning_rate),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    # Model summary
    model.summary()

    # Training callbacks
    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_accuracy',
            patience=10,
            restore_best_weights=True,
            min_delta=0.005
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.1,
            patience=5,
            min_lr=1e-6
        ),
        tf.keras.callbacks.ModelCheckpoint(
            'best_model.h5',
            monitor='val_accuracy',
            save_best_only=True
        )
    ]

    # Train the model
    epochs = 50
    history = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=epochs,
        callbacks=callbacks
    )

    # Fine-tune the model
    base_model.trainable = True
    fine_tune_epochs = 20
    total_epochs = epochs + fine_tune_epochs

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(),
        metrics=['accuracy']
    )

    history_fine_tune = model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=total_epochs,
        initial_epoch=history.epoch[-1],
        callbacks=callbacks
    )

    # Save the final model
    model.save('final_model.h5')

    print("Training completed!")
    print(f"Final training accuracy: {history_fine_tune.history['accuracy'][-1]:.4f}")
    print(f"Final validation accuracy: {history_fine_tune.history['val_accuracy'][-1]:.4f}")

if __name__ == "__main__":
    main()

Found 2250 files belonging to 3 classes.
Using 1800 files for training.
Found 2250 files belonging to 3 classes.
Using 450 files for validation.
Classes: ['paper', 'rock', 'scissors']


Model: "functional_205"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ input_layer_7 (InputLayer)           │ (None, 224, 224, 3)         │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ efficientnetb0 (Functional)          │ (None, 7, 7, 1280)          │       4,049,571 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ global_average_pooling2d_1           │ (None, 1280)                │               0 │
│ (GlobalAveragePooling2D)             │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_14 (Dense)                     │ (None, 512)                 │         655,872 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_9 (Dropout)                  │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_15 (Dense)                     │ (None, 3)                   │           1,539 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 4,706,982 (17.96 MB)

 Trainable params: 657,411 (2.51 MB)

...

29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 99ms/step - accuracy: 0.9510 - loss: 0.1189 - val_accuracy: 0.9622 - val_loss: 0.0972 - learning_rate: 0.0010
Epoch 6/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9703 - loss: 0.0878

29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 90ms/step - accuracy: 0.9702 - loss: 0.0879 - val_accuracy: 0.9733 - val_loss: 0.0816 - learning_rate: 0.0010
Epoch 7/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9535 - loss: 0.1154

29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 85ms/step - accuracy: 0.9540 - loss: 0.1146 - val_accuracy: 0.9756 - val_loss: 0.0820 - learning_rate: 0.0010
Epoch 8/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9769 - loss: 0.0696

29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 89ms/step - accuracy: 0.9768 - loss: 0.0698 - val_accuracy: 0.9800 - val_loss: 0.0697 - learning_rate: 0.0010
Epoch 9/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 72ms/step - accuracy: 0.9743 - loss: 0.0680 - val_accuracy: 0.9689 - val_loss: 0.0776 - learning_rate: 0.0010
Epoch 10/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - accuracy: 0.9745 - loss: 0.0691 - val_accuracy: 0.9667 - val_loss: 0.0848 - learning_rate: 0.0010
Epoch 11/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.9692 - loss: 0.0786 - val_accuracy: 0.9756 - val_loss: 0.0641 - learning_rate: 0.0010
Epoch 12/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.9674 - loss: 0.0812 - val_accuracy: 0.9667 - val_loss: 0.0845 - learning_rate: 0.0010
Epoch 13/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 3s 73ms/step - accuracy: 0.9802 - loss: 0.0467 - val_accuracy: 0.9778 - val_loss: 0.0675 - learning_rate: 0.0010
Epoch 14/50
29/29 ━━━━━━━━━━━━━━━━━━━━ 2s 73ms/step - accuracy: 0.9828 - loss: 0.0524 - val_accuracy:

Training completed!
Final training accuracy: 0.9494
Final validation accuracy: 0.9489
